In [11]:
# Update paths & load required packages
path2code = '/script_dir/'
path2data = '/data_dir/'
path2raw = path2data + 'raw/'
path2clean = path2data + 'clean/'

import datetime, mmap, os, re, sys, decimal, glob
from datetime import datetime
from dateutil import parser
import numpy as np 
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt 
import rpy2
from collections import defaultdict
from scipy.stats import pearsonr 
%load_ext rpy2.ipython
%matplotlib inline 

In [13]:
# Load all surveys into one dataframe. Each subject has up to 6 .CSVs, 1 for every 2-hour time window (i.e. 10 AM, 12 PM).
dfs = []
out = pd.DataFrame

files = (glob.glob(path2raw + '*.csv'))
for file in files:
    df = pd.read_csv(file, skiprows = [0,2])
    df['survey_number'] = file[-5]
    dfs.append(df)
out = pd.DataFrame
out = pd.concat(dfs, sort=False)

In [14]:
# Pull in header names and rename headers
headers = pd.read_csv('headers.csv')

old_headers = headers['original']
new_headers = headers['new']
rename_dict = dict(zip(old_headers, new_headers))

out.rename(columns=rename_dict, inplace=True)

In [15]:
# Check headers
out.head()

,start_date,end_date,response_type,progress,duration,finished,recorded_date,responseID,dist_channel,language,...,anhedonia_day,down_day,stressed_day,tired_day,nervous_day,uncontrolled_worry_day,worry_day,low_high_mood_day,survey_number,sleep_quality
0,2017-11-14 18:15:29,2017-11-14 18:19:44,0,100,254,1,2017-11-14 18:19:44,R_2ZTsOaNZSwZMekF,anonymous,EN,...,3.0,2.0,2.0,2.0,2.0,2.0,1.0,3.0,6,NaN
1,2017-11-15 22:14:46,2017-11-15 22:36:02,0,100,1275,1,2017-11-15 22:36:02,R_3rG9kxKEPs5gs60,anonymous,EN,...,3.0,3.0,4.0,3.0,3.0,3.0,3.0,3.0,6,NaN
2,2017-11-16 19:36:08,2017-11-16 19:40:28,0,100,260,1,2017-11-16 19:40:28,R_4ONLvitXRz34yyt,anonymous,EN,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4.0,6,NaN
3,2017-11-16 20:47:37,2017-11-16 20:51:21,0,100,224,1,2017-11-16 20:51:21,R_3J7W9JQY4Sjiyva,anonymous,EN,...,1.0,1.0,2.0,1.0,2.0,2.0,1.0,1.0,6,NaN
4,2017-11-17 22:13:50,2017-11-17 22:17:27,0,100,216,1,2017-11-17 22:17:27,R_OEakzsMQTbHmVBn,anonymous,EN,...,2.0,2.0,3.0,2.0,3.0,2.0,2.0,4.0,6,NaN


In [17]:
# Identify unique subject IDs, based on subject report
out['subjectID'] = out.subjectID.apply(lambda x: int(str(x).replace('-','')))
out.subjectID.unique()

array([ 1016,  1011,  1005,  1017,  1015,  1007,  1012,  1020,  1019,
        1022,  1024,  1026,  1023,  1025,  1027,  1029,  1100,  1034,
        1033,  1101,  1102,  1032,  1103,  1104,  1105,  1038,  9999,
        1107,  1037,  1109,  1044,  1108,  1110,  1045,  1043,  1111,
        1042,  1114,  1116,  1117,  1120,  1118,  1122,  1121,  1125,
        1127,  1128,  1132,  1126,  1130,  1131,  1133,  1145,  1136,
        1135,  1138,  1137,  1141,  1142,   100,   119,  1106,  1150,
        1146,  1013,  1036,  1123,     0,  1139, 11038,  1003,  2043,
         114,  1035,  1112, 11117])

In [18]:
# Correct misentered subject IDs with correct subject IDs from subject log
realsubs = pd.read_csv('subjects.csv')

realsubs['subjects'] = realsubs.subjects.apply(lambda x: str(x))
out = out.loc[out.subjectID.isin(realsubs.subjects.tolist())]

print(out.subjectID.unique())

print('\nTotal subjects:',len(out.subjectID.unique()))

array([1100, 1101, 1102, 1103, 1104, 1105, 1107, 1109, 1110, 1111, 1114,
       1116, 1117, 1118, 1120, 1122, 1121, 1125, 1127, 1128, 1126, 1130,
       1131, 1132, 1133, 1005, 1007, 1011, 1012, 1015, 1016, 1017, 1019,
       1020, 1022, 1023, 1024, 1025, 1026, 1027, 1029, 1032, 1033, 1034,
       1038, 1037, 1044, 1043, 1042, 1135, 1136, 1138, 1137, 1141])

In [10]:
# Remove subjects that did not comply with EMA protocol, based on study notes
out = out[out.subjectID != 1100]
out = out[out.subjectID != 1101]

In [11]:
# Exclude subjects with less than 3 surveys & identify subjects with less than 20 surveys
out['outlier_surveynum'] = 0
for subject in out.subjectID.unique():
    if len(out.loc[out.subjectID == subject]) < 3:
        print(subject, 'was excluded for having ' + str(len(out[out.subjectID == subject])) + ' survey(s)')
        out = out[out.subjectID != subject]
    elif len(out[out.subjectID == subject]) < 20:
        print(subject, 'was kept in but only has ' + str(len(out[out.subjectID == subject])) + ' surveys')
        out.loc[out.subjectID == subject, 'outlier_surveynum'] = 1

1131 was kept in but only has 8 surveys


In [12]:
# 52 subects remain
print(out.subjectID.unique())
print('\nNumber of subjects: ', (out.subjectID.nunique()))

[1016 1011 1005 1017 1015 1007 1012 1020 1019 1022 1024 1026 1023 1025
 1027 1029 1034 1033 1102 1032 1103 1104 1105 1038 1107 1037 1109 1044
 1110 1043 1111 1042 1114 1116 1117 1120 1118 1122 1121 1125 1127 1128
 1132 1126 1130 1131 1133 1136 1135 1138 1137 1141]

Number of subjects:  52


In [14]:
# Identify outliers based on time it took to complete the survey, to be excluded later
out['outlier_time'] = 0
out.loc[(out.duration < 30)|(out.duration > 86400),'outlier_time'] = 1

out.outlier_time.value_counts()

0    3837
1      47
Name: outlier_time, dtype: int64

In [18]:
# Identify and exclude unfinished surveys
out.finished.value_counts()

out = out.loc[(out.finished == 1)]

In [15]:
# Convert variables to date format
out['end_date'] = pd.to_datetime(out.end_date)
out['start_date'] = pd.to_datetime(out.start_date)

In [19]:
# Create variable to identify the survey number based on time of day it was completed
out['survey_number'] = 0

df = pd.DataFrame()
for sub in out.subjectID.unique():
    data = out[out.subjectID == sub]
    data = data.sort_values(by = ['end_date'])
    data = data.reset_index(drop=True)
    for i, row_i in data.iterrows():
        if i < 1:
            data.loc[i, 'survey_number'] = 1
        elif row_i.end_date.date() == data.loc[i - 1].end_date.date():
            data.loc[i, 'survey_number'] = (data.loc[i - 1, 'survey_number'] + 1)
        else:
            data.loc[i, 'survey_number'] = 1
    df = pd.concat([data, df])

out = df

In [20]:
# Verifying number of surveys
out.survey_number.unique()
out.survey_number.value_counts()
out.loc[out.survey_number > 6][['subjectID','end_date','duration','survey_number']]

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [23]:
# Remove all surveys after survey 6
out = out[out.survey_number < 7]
out.survey_number.unique()

array([1, 2, 3, 4, 5, 6])

In [24]:
# Identify time between each survey based on times they were completed
df = pd.DataFrame()
out['time_between'] = np.nan

data = out
data = data.reset_index(drop=True)
for i, row_i in data.iterrows():
    if row_i.survey_number == 1:
        pass
    else:
        diff = row_i.end_date - (data.loc[i - 1, 'end_date'])
        data.loc[i, 'time_between'] = diff

data.time_between = data.time_between.apply(lambda x: np.nan if pd.isnull(x) else x.total_seconds()/3600)
data[['subjectID','end_date','duration','survey_number','time_between']].head(10)

,subjectID,end_date,duration,survey_number,time_between
0,1141,2019-05-01 10:07:43,288,1,NaN
1,1141,2019-05-01 12:11:43,323,2,0 days 02:04:00
2,1141,2019-05-01 14:04:34,228,3,0 days 01:52:51
3,1141,2019-05-01 16:09:08,252,4,0 days 02:04:34
4,1141,2019-05-01 18:05:18,265,5,0 days 01:56:10
5,1141,2019-05-01 20:59:07,417,6,0 days 02:53:49
6,1141,2019-05-02 14:34:20,1974,1,NaN
7,1141,2019-05-03 10:18:14,975,1,NaN
8,1141,2019-05-03 12:07:29,280,2,0 days 01:49:15
9,1141,2019-05-03 15:52:15,6681,3,0 days 03:44:46


In [27]:
# Flag the survey as an outlier if the survey BEFORE it was completed less than 1 hour or greater than 3, for exclusion later on
data['outlier_timebefore'] = 0
data.loc[(data.time_between < 1)|(data.time_between >3), 'outlier_timebefore'] = 1

print(data.outlier_timebefore.value_counts())

print('\nNumber of surveys with the previous one completed > 3 hours ago: '+ str(len(data.loc[(data.time_between > 3)])))
print('\nNumber of surveys with the previous one completed < 1 hour ago: '+ str(len(data.loc[(data.time_between < 1)])))

In [33]:
# Create day variable to show number of days since the start of EMA protocol
df = pd.DataFrame()
data['day'] = 0
for sub in data.subjectID.unique():
    tst_data = data[data.subjectID == sub]
    tst_data = tst_data.sort_values(by = ['end_date'])
    tst_data = tst_data.reset_index(drop=True)
    for i, row_i in tst_data.iterrows():
        if i < 1:
            start = row_i.end_date.date()
            tst_data.loc[i, 'day'] = 1
        elif row_i.end_date.date() == start:
            tst_data.loc[i, 'day'] = 1
        else:
            tst_data.loc[i, 'day'] = row_i.end_date.date() - start
    df = pd.concat([tst_data, df])
    
df.day = df.day.apply(lambda x: x if isinstance(x, int) else x.total_seconds()/86400 + 1)

In [37]:
# Create PANAS positive and negative affect scores
df['PA'] = df['enthusiastic_current'] + df['cheerful_current'] + df['relaxed_current']
df['NA'] = df['irritable_current'] + df['anxious_current'] + df['sad_current']

In [40]:
# Create new dataframe and add clinical group variable
data = df
data['group'] = 'CTL'
data.loc[data.subjectID < 1100, 'group'] = 'MDD'

**Note on prediction error variables**  
A PE is only calculated if...  
1) a valid prediction was made within 1 - 3 hours prior and not first survey of the day),  
2) the current survey was not a time outlier, and  
3) the prior survey was not a time outlier  

In [47]:
# Create PE and expectation variables. Flag PE outliers based on number of surveys and time between.
data['expectation'] = np.nan
data['PE'] = np.nan
data['PE_exclude'] = 0

data = data.reset_index(drop=True)
for i, row_i in data.iterrows():
    if row_i.survey_number == 1: # can't get a PE for the first day
        pass
    else:
        prev = data.loc[i - 1, 'expected_value'] # get the prediction from the previous row
        data.loc[i, 'expectation'] = prev
        if ((row_i.outlier_timebefore == 1) | (row_i.outlier_surveynum == 1) | (row_i.outlier_time ==1)):
            data.loc[i, 'PE_exclude'] = 1
        else:
            pass
        
data['PE'] = data['actual_value'] - data['expectation'] # create a prediction error

data[['subjectID','end_date','duration','survey_number','expected_value',\
      'actual_value', 'PE']].head(10)

,subjectID,end_date,duration,survey_number,expected_value,expectation,actual_value,PE,expected_value_confidence,confidence_prev
0,1016,2017-11-14 08:15:09,271,1,2.0,NaN,2.0,NaN,80.0,NaN
1,1016,2017-11-14 10:09:30,174,2,-1.0,2.0,0.0,-2.0,85.0,80.0
2,1016,2017-11-14 12:25:29,147,3,3.0,-1.0,0.0,1.0,80.0,85.0
3,1016,2017-11-14 14:31:13,106,4,4.0,3.0,4.0,1.0,100.0,80.0
4,1016,2017-11-14 17:58:04,205,5,1.0,4.0,2.0,-2.0,80.0,100.0
5,1016,2017-11-14 18:19:44,254,6,1.0,1.0,2.0,1.0,90.0,80.0
6,1016,2017-11-16 08:32:46,151,1,3.0,NaN,3.0,NaN,80.0,NaN
7,1016,2017-11-16 10:32:37,81,2,4.0,3.0,4.0,1.0,100.0,80.0
8,1016,2017-11-16 12:21:55,281,3,4.0,4.0,4.0,0.0,100.0,100.0
9,1016,2017-11-16 14:33:33,158,4,4.0,4.0,NaN,NaN,100.0,100.0


In [51]:
# Remove subjects flagged for exclusion and store in a new, clean dataframe
clean = data[data.PE_exclude == 0]

for subject in clean.subjectID.unique():
    if len(clean.loc[clean.subjectID == subject]) < 10:
        print(subject, ' was excluded for having ' + str(len(clean[clean.subjectID == subject])) + ' survey(s)')
        clean = clean[clean.subjectID != subject]

1038  was excluded for having 9 survey(s)
1131  was excluded for having 3 survey(s)


In [ ]:
# Create absolute value, negative, and positive PE, EV, and AV variables.

# Create PE_abs variable
clean['PE_abs'] = clean.PE.abs()

# Create negative PE
clean['neg_PE'] = np.nan
clean.loc[clean.PE < 0, 'neg_PE'] = clean.PE

# Create positive PE
clean['pos_PE'] = np.nan
clean.loc[clean.PE > 0, 'pos_PE'] = clean.PE

# Create frequencies of different PE types
clean['pos_PE_freq'] = np.nan
clean['neg_PE_freq'] = np.nan
clean['no_PE_freq'] = np.nan
for subject in clean.subjectID.unique():
    sub_data = clean.loc[clean.subjectID == subject]
    sub_data.reset_index(drop=True)
    neg_PE = len(sub_data[sub_data.PE < 0])
    pos_PE = len(sub_data[sub_data.PE > 0])
    no_PE = len(sub_data[sub_data.PE == 0])
    PE_tot = neg_PE + pos_PE + no_PE
    clean.loc[clean.subjectID == subject, 'pos_PE_freq'] = pos_PE/PE_tot
    clean.loc[clean.subjectID == subject, 'neg_PE_freq'] = neg_PE/PE_tot
    clean.loc[clean.subjectID == subject, 'no_PE_freq'] = no_PE/PE_tot

In [ ]:
#Create average PA, NA, and PE variables
clean['average_PA'] = np.nan
clean['average_NA'] = np.nan
clean['average_PE'] = np.nan

for sub in clean.subjectID.unique():
    average_PE = clean[clean.subjectID == sub].PE.mean()
    average_PA = clean[clean.subjectID == sub].PA.mean()
    average_NA = clean[clean.subjectID == sub].NA.mean()
    clean.loc[clean.subjectID==sub, 'average_PE'] = average_PE
    clean.loc[clean.subjectID==sub, 'average_PA'] = average_PA
    clean.loc[clean.subjectID==sub, 'average_NA'] = average_NA

In [ ]:
#Create frequency of negative, positive, and accurate/zero PE variables and assign them to a PE category
clean['PE_category'] = ''
clean['neg_PE_ct'] = 0
clean['pos_PE_ct'] = 0
clean['zero_PE_ct'] = 0

for subject in clean.subjectID.unique():
    sub_data = clean[clean.subjectID == subject]
    
    neg_ct = len(sub_data[sub_data.PE < 0])
    pos_ct = len(sub_data[sub_data.PE > 0])
    zero_ct = len(sub_data[sub_data.PE == 0])
    
    clean.loc[clean.subjectID == subject, 'pos_PE_ct'] = pos_ct
    clean.loc[clean.subjectID == subject, 'neg_PE_ct'] = neg_ct
    clean.loc[clean.subjectID == subject, 'zero_PE_ct'] = zero_ct
    
    if (neg_ct > pos_ct) & (neg_ct > zero_ct):
        clean.loc[clean.subjectID == subject, 'PE_category'] = 'over'
    elif (pos_ct > neg_ct) & (pos_ct > zero_ct):
        clean.loc[clean.subjectID == subject, 'PE_category'] = 'under'
    elif (zero_ct > pos_ct) & (zero_ct > neg_ct):
        clean.loc[clean.subjectID == subject, 'PE_category'] = 'zero'
    else:
        print(str(subject) + ' has a tie')

In [ ]:
# Normalize the positive and negative PEs by dividing by the number of accurate predictions (i.e. zero PE)
clean['pos_zero_PE_diff'] = clean.pos_PE_ct/clean.zero_PE_ct
clean['neg_zero_PE_diff'] = clean.neg_PE_ct/clean.zero_PE_ct

In [ ]:
#Save final, clean data to file
clean.to_csv(path2clean + 'EMA_Final.csv')